In [1]:
import geopandas as gpd
import pandas as pd

# Load the pedon and petaxhistory layers
gdb_path = "nasispedonsAK.gdb"
pedon_gdf = gpd.read_file(gdb_path, layer="pedon")
petaxhistory_gdf = gpd.read_file(gdb_path, layer="petaxhistory")

# Extract the geometry components
pedon_gdf['geom_lon'] = pedon_gdf.geometry.x  # Extracts longitude
pedon_gdf['geom_lat'] = pedon_gdf.geometry.y  # Extracts latitude

# Count occurrences of peiidref in petaxhistory
peiid_counts = (petaxhistory_gdf['peiidref']
                .value_counts()
                .reset_index(name='count'))

# Rename the columns for clarity
peiid_counts.columns = ['peiid', 'record_count']

# Filter for peiid values with multiple records
multiple_records = peiid_counts[peiid_counts['record_count'] > 1].copy()

# Create a list of record numbers for each peiid
multiple_records['record_numbers'] = multiple_records['peiid'].apply(
    lambda x: petaxhistory_gdf[petaxhistory_gdf['peiidref'] == x].index.tolist()
)

# Final DataFrame with desired information
final_df = multiple_records[['peiid', 'record_count', 'record_numbers']].copy()

# Initialize lists to hold new data
recent_years = []
soiltaxeditions = []
tax_orders = []
tax_suborders = []

# Iterate through each peiid in final_df
for peiid in final_df['peiid']:
    # Filter petaxhistory for the current peiid
    relevant_records = petaxhistory_gdf[petaxhistory_gdf['peiidref'] == peiid].copy()
    
    if not relevant_records.empty:
        # Convert 'class date' to datetime
        relevant_records['class_date'] = pd.to_datetime(relevant_records['classdate'])
        
        # Find the most recent record
        most_recent_index = relevant_records['class_date'].idxmax()
        most_recent_record = relevant_records.loc[most_recent_index]
        
        # Get the year, soiltaxedition, tax order, and tax suborder
        recent_years.append(most_recent_record['class_date'].year)
        soiltaxeditions.append(most_recent_record['soiltaxedition'])
        tax_orders.append(most_recent_record['taxorder'])
        tax_suborders.append(most_recent_record['taxsuborder'])
    else:
        recent_years.append(None)
        soiltaxeditions.append(None)
        tax_orders.append(None)
        tax_suborders.append(None)

# Use .loc to append new columns to final_df
final_df.loc[:, 'most_recent_year'] = recent_years
final_df.loc[:, 'soiltaxedition'] = soiltaxeditions
final_df.loc[:, 'tax_order'] = tax_orders
final_df.loc[:, 'tax_suborder'] = tax_suborders

# Now merge the latitude and longitude from the pedon_gdf
final_df = final_df.merge(
    pedon_gdf[['peiid', 'geom_lon', 'geom_lat']],
    how='left',
    left_on='peiid',
    right_on='peiid'
)

# Inspect the result
print(final_df.head())


    peiid  record_count                       record_numbers  \
0  741425             5  [23193, 23194, 23195, 23196, 23197]   
1  741426             5  [23198, 23199, 23200, 23201, 23202]   
2  741444             5  [23278, 23279, 23280, 23281, 23282]   
3  843706             5       [9174, 9175, 9176, 9177, 9178]   
4  741438             5  [23253, 23254, 23255, 23256, 23257]   

   most_recent_year      soiltaxedition  tax_order tax_suborder    geom_lon  \
0              2023  thirteenth edition  Spodosols       Cryods -152.398856   
1              2023  thirteenth edition  Spodosols       Cryods -152.397464   
2              2023                None       None         None -152.403986   
3              2020     twelfth edition   Entisols      Aquents -145.027910   
4              2024  thirteenth edition  Spodosols       Cryods -152.405861   

    geom_lat  
0  57.805808  
1  57.806583  
2  57.812019  
3  66.954290  
4  57.806497  


In [2]:
# write to csv, index=False: This prevents pandas from writing row indices to the CSV file, which is often preferable.
final_df.to_csv('final_data.csv', index=False)